In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states

# Census & gmaps API Keys
from config import api_key_census
c = Census(api_key_census, year=2013)

#  Configure gmaps
#gmaps.configure(api_key=gkey)

In [4]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'zip code tabulation area:*'})


# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", 
                                      "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]

census_pd.head()

,Zipcode,Population,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate
0,08518,5217.0,74286.0,33963.0,170.0,3.258578,6.152961
1,08520,27468.0,90293.0,37175.0,1834.0,6.676860,5.162371
2,08525,4782.0,118656.0,59848.0,43.0,0.899205,2.885822
3,08527,54867.0,88588.0,37021.0,2191.0,3.993293,5.303734
4,08528,245.0,58676.0,49117.0,0.0,0.000000,0.000000
